In [1]:
import numpy as np
import category_encoders
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Limpieza de datos

Eliminar columnas que tienen mas de la mitad de sus datos vacios

In [3]:
clean_train = train.drop(columns = ['Last_Activity','Actual_Delivery_Date','Price','Size','Product_Type','Brand', 'Product_Category_B', 'Source '])
clean_test = test.drop(columns = ['Last_Activity','Actual_Delivery_Date','Price','Size','Product_Type','Brand', 'Product_Category_B', 'Source '])

Elimanar columnas que no me aportan nada como la columna que provoca 'target leakage' 

In [4]:
clean_train = clean_train.drop(columns = ['Sales_Contract_No', 'Opportunity_Name'])
clean_test = clean_test.drop(columns = ['Sales_Contract_No', 'Opportunity_Name'])
#FALTA ELIMINAR EL OPPORTUNITY_ID pero lo necesitamos para agrupar y luego armar el submit

Filtrar registros que esten en estado finalizado y conversion en valores binarios  
Closed Won = 1 - Closed Lost = 0

In [5]:
clean_train = clean_train[(clean_train['Stage'] == 'Closed Won') | (clean_train['Stage'] == 'Closed Lost')]
clean_train['Stage'] = clean_train['Stage'].transform(func=lambda x : 1 if x=='Closed Won' else 0)

Reemplazar los datos vacios con "Nan"

In [6]:
clean_train = clean_train.fillna("Nan")
clean_test = clean_test.fillna("Nan")

# Creacion de features

In [7]:
features_train = clean_train.copy()
features_test = clean_test.copy()

Convierto columnas con fecha a date

In [8]:
features_train["Last_Modified_Date"] = pd.to_datetime(features_train["Last_Modified_Date"], errors='coerce')
features_train["Opportunity_Created_Date"] = pd.to_datetime(features_train["Opportunity_Created_Date"], errors='coerce')
features_train["Quote_Expiry_Date"] = pd.to_datetime(features_train["Quote_Expiry_Date"], errors='coerce')
features_train["Planned_Delivery_Start_Date"] = pd.to_datetime(features_train["Planned_Delivery_Start_Date"], errors='coerce')
features_train["Planned_Delivery_End_Date"] = pd.to_datetime(features_train["Planned_Delivery_End_Date"], errors='coerce')

features_test["Last_Modified_Date"] = pd.to_datetime(features_test["Last_Modified_Date"], errors='coerce')
features_test["Opportunity_Created_Date"] = pd.to_datetime(features_test["Opportunity_Created_Date"], errors='coerce')
features_test["Quote_Expiry_Date"] = pd.to_datetime(features_test["Quote_Expiry_Date"], errors='coerce')
features_test["Planned_Delivery_Start_Date"] = pd.to_datetime(features_test["Planned_Delivery_Start_Date"], errors='coerce')
features_test["Planned_Delivery_End_Date"] = pd.to_datetime(features_test["Planned_Delivery_End_Date"], errors='coerce')

Agrego columna Diferencia en dias entre la fecha de creacion de la oportunidad y la ultima modificacion de la oportunidad

In [9]:
features_train["diferencia_en_dias"] = (features_train["Last_Modified_Date"] - features_train["Opportunity_Created_Date"]).dt.days

features_test["diferencia_en_dias"] = (features_test["Last_Modified_Date"] - features_test["Opportunity_Created_Date"]).dt.days

Casteo variables numericas

In [10]:
features_train["ASP_(converted)"] = pd.to_numeric(features_train["ASP_(converted)"],errors='coerce')
features_train["ASP"] = pd.to_numeric(features_train["ASP"],errors='coerce')
features_train["TRF"] = pd.to_numeric(features_train["TRF"],errors='coerce')
features_train["Total_Amount"] = pd.to_numeric(features_train["Total_Amount"],errors='coerce')
features_train["Total_Taxable_Amount"] = pd.to_numeric(features_train["TRF"],errors='coerce')

In [11]:
features_test["ASP_(converted)"] = pd.to_numeric(features_test["ASP_(converted)"],errors='coerce')
features_test["ASP"] = pd.to_numeric(features_test["ASP"],errors='coerce')
features_test["TRF"] = pd.to_numeric(features_test["TRF"],errors='coerce')
features_test["Total_Amount"] = pd.to_numeric(features_test["Total_Amount"],errors='coerce')
features_test["Total_Taxable_Amount"] = pd.to_numeric(features_test["TRF"],errors='coerce')

Division variables de tipo fecha en DOY, año

In [12]:
features_train["Last_Modified_DOY"] = features_train['Last_Modified_Date'].dt.dayofyear
features_train["Last_Modified_Year"] = features_train['Last_Modified_Date'].dt.year

features_train["Opportunity_Created_DOY"] = features_train["Opportunity_Created_Date"].dt.dayofyear
features_train["Opportunity_Created_Year"] = features_train["Opportunity_Created_Date"]

features_train["Quote_Expiry_DOY"] = features_train["Quote_Expiry_Date"].dt.dayofyear
features_train["Quote_Expiry_Year"] = features_train["Quote_Expiry_Date"].dt.year

features_train["Planned_Delivery_Start_DOY"] = features_train["Planned_Delivery_Start_Date"].dt.dayofyear
features_train["Planned_Delivery_Start_Year"] = features_train["Planned_Delivery_Start_Date"].dt.year

features_train["Planned_Delivery_End_DOY"] = features_train["Planned_Delivery_End_Date"].dt.dayofyear
features_train["Planned_Delivery_End_Year"] = features_train["Planned_Delivery_End_Date"].dt.year 


features_train.drop(columns = ['Planned_Delivery_End_Date', 'Planned_Delivery_Start_Date','Quote_Expiry_Date','Opportunity_Created_Date','Last_Modified_Date'], inplace=True)

features_train["Quote_Expiry_DOY"] = features_train["Quote_Expiry_DOY"].fillna('Nan')
features_train["Quote_Expiry_Year"] = features_train["Quote_Expiry_Year"].fillna('Nan')
features_train["Planned_Delivery_End_DOY"] = features_train["Planned_Delivery_End_DOY"].fillna('Nan')
features_train["Planned_Delivery_End_Year"] = features_train["Planned_Delivery_End_Year"].fillna('Nan')

In [13]:
features_test["Last_Modified_DOY"] = features_test['Last_Modified_Date'].dt.dayofyear
features_test["Last_Modified_Year"] = features_test['Last_Modified_Date'].dt.year

features_test["Opportunity_Created_DOY"] = features_test["Opportunity_Created_Date"].dt.dayofyear
features_test["Opportunity_Created_Year"] = features_test["Opportunity_Created_Date"]

features_test["Quote_Expiry_DOY"] = features_test["Quote_Expiry_Date"].dt.dayofyear
features_test["Quote_Expiry_Year"] = features_test["Quote_Expiry_Date"].dt.year

features_test["Planned_Delivery_Start_DOY"] = features_test["Planned_Delivery_Start_Date"].dt.dayofyear
features_test["Planned_Delivery_Start_Year"] = features_test["Planned_Delivery_Start_Date"].dt.year

features_test["Planned_Delivery_End_DOY"] = features_test["Planned_Delivery_End_Date"].dt.dayofyear
features_test["Planned_Delivery_End_Year"] = features_test["Planned_Delivery_End_Date"].dt.year 

features_test.drop(columns = ['Planned_Delivery_End_Date', 'Planned_Delivery_Start_Date','Quote_Expiry_Date','Opportunity_Created_Date','Last_Modified_Date'], inplace=True)

features_test["Quote_Expiry_DOY"] = features_test["Quote_Expiry_DOY"].fillna('Nan')
features_test["Quote_Expiry_Year"] = features_test["Quote_Expiry_Year"].fillna('Nan')
features_test["Planned_Delivery_End_DOY"] = features_test["Planned_Delivery_End_DOY"].fillna('Nan')
features_test["Planned_Delivery_End_Year"] = features_test["Planned_Delivery_End_Year"].fillna('Nan')

Agrego columnas como concatenacion de columnas

In [14]:
features_train["Delivery_Quarter_of_Year"] = features_train["Delivery_Quarter"] +"-"+ (features_train["Delivery_Year"].astype(str))

In [15]:
features_test["Delivery_Quarter_of_Year"] = features_test["Delivery_Quarter"] +"-"+ (features_test["Delivery_Year"].astype(str))

In [16]:
features_train["Region_Territory_Country"] = features_train["Region"] +" "\
+ features_train["Territory"]+" "\
+ features_train["Billing_Country"]

In [17]:
features_test["Region_Territory_Country"] = features_test["Region"] +" "\
+ features_test["Territory"]+" "\
+ features_test["Billing_Country"]

In [18]:
features_train["Product_Family_Name"] = features_train["Product_Family"] +"-"+ features_train["Product_Name"]

In [19]:
features_test["Product_Family_Name"] = features_test["Product_Family"] +"-"+ features_test["Product_Name"]

Agrego columna binaria si el asp es mayor al promedio o no (uso el convertido por el tipo de moneda)

In [20]:
#Obtengo el promedio eliminando las 2 filas que tenian valores mal
features_train.loc[:,["Opportunity_ID","ASP_(converted)"]].sort_values(by='ASP_(converted)',ascending=False).head()
asp_mean_train = features_train.drop([1177, 1164])["ASP_(converted)"].mean()
#Lleno las 9 filas del set que no tienen datos de ASP con el promedio
features_train["ASP"] = features_train['ASP'].fillna(asp_mean_train)
features_train["ASP_(converted)"] = features_train['ASP_(converted)'].fillna(asp_mean_train)
#Coloco 1 si es mayor al promedio 0 en caso contrario
features_train["ASP_Higher_Mean"] = features_train['ASP_(converted)'].transform(func=lambda x : 1 if x >= asp_mean_train else 0)

In [21]:
#Obtengo el promedio eliminando las 2 filas que tenian valores mal
features_test.loc[:,["Opportunity_ID","ASP_(converted)"]].sort_values(by='ASP_(converted)',ascending=False).head()
asp_mean_test = features_test.drop([810])["ASP_(converted)"].mean()
features_test["ASP"] = features_test['ASP'].fillna(asp_mean_test)
features_test["ASP_(converted)"] = features_test['ASP_(converted)'].fillna(asp_mean_test)
#Coloco 1 si es mayor al promedio 0 en caso contrario
features_test["ASP_Higher_Mean"] = features_test['ASP_(converted)'].transform(func=lambda x : 1 if x >= asp_mean_test else 0)

Agrego columna binaria si las toneladas de frigorias de la oportunidad es mayor al promedio

In [22]:
trf_mean_train = features_train["TRF"].mean()
#Coloco 1 si es mayor al promedio 0 en caso contrario
features_train["TRF_Higher_Mean"] = features_train['TRF'].transform(func=lambda x : 1 if x >= trf_mean_train else 0)

In [23]:
trf_mean_test = features_test["TRF"].mean()
#Coloco 1 si es mayor al promedio 0 en caso contrario
features_test["TRF_Higher_Mean"] = features_test['TRF'].transform(func=lambda x : 1 if x >= trf_mean_test else 0)

Agrego columna binaria si el responsable de cuenta es el mismo que el responsable de oportunidad

In [24]:
features_train["Same_Owner"] = features_train["Account_Owner"] == features_train["Opportunity_Owner"]
features_train["Same_Owner"] = features_train["Same_Owner"].transform(func=lambda x : 1 if x else 0)

In [25]:
features_test["Same_Owner"] = features_test["Account_Owner"] == features_test["Opportunity_Owner"]
features_test["Same_Owner"] = features_test["Same_Owner"].transform(func=lambda x : 1 if x else 0)

Agrego columna binaria si el responsable de cuenta es el ultimo que modifico oportunidad

In [26]:
features_train["Last_Modified_By_Account_Owner"] = features_train["Account_Owner"] == features_train["Last_Modified_By"]
features_train["Last_Modified_By_Account_Owner"] = features_train["Last_Modified_By_Account_Owner"].transform(func=lambda x : 1 if x else 0)

In [27]:
features_test["Last_Modified_By_Account_Owner"] = features_test["Account_Owner"] == features_test["Last_Modified_By"]
features_test["Last_Modified_By_Account_Owner"] = features_test["Last_Modified_By_Account_Owner"].transform(func=lambda x : 1 if x else 0)

Agrego columna binaria si el responsable de oportunidad es el ultimo que modifico oportunidad

In [28]:
features_train["Last_Modified_By_Opportunity_Owner"] = features_train["Opportunity_Owner"] == features_train["Last_Modified_By"]
features_train["Last_Modified_By_Opportunity_Owner"] = features_train["Last_Modified_By_Opportunity_Owner"].transform(func=lambda x : 1 if x else 0)

In [29]:
features_test["Last_Modified_By_Opportunity_Owner"] = features_test["Opportunity_Owner"] == features_test["Last_Modified_By"]
features_test["Last_Modified_By_Opportunity_Owner"] = features_test["Last_Modified_By_Opportunity_Owner"].transform(func=lambda x : 1 if x else 0)

Dejar la columna Stage al final 

In [30]:
features_train["Target"] = features_train["Stage"]
features_train.drop(columns = ['Stage'], inplace=True)

In [31]:
features_train.to_csv('data/other-cleaned_train.csv')
features_test.to_csv('data/other-cleaned_test.csv')